# ITCH trade sign classification (explanation)

#### Juan Camilo Henao Londono - 27.12.2019
#### AG Guhr - Universität Duisburg-Essen

The results of the trade sign classification from the [paper](https://link.springer.com/content/pdf/10.1140/epjb/e2016-60818-y.pdf) are different from the ones I obtained with my implementation.
In this notebook I want to clarify the reasons why they are different. I think my code is right and the mistake
is in the implementation and interpretation in the paper.

In [1]:
import gzip
import numpy as np
import pandas as pd

import itch_trade_sign_classification_test as itch_sign_clas

## Part 1. Original results

To compare in detail the results I use the TotalView-ITCH data for the Apple Inc. stock the 2008.06.02.
The paper uses Eq. 1, 2 and 3 in Section 2.3 to obtain the trade signs. I will only use the data obtained from Eq. 1 and 2 to clarify my position. Eq. 1 uses trade time scale and Eq.2 uses second time scale.

In [2]:
# Ticker and date
ticker = 'AAPL'
year = '2008'
month = '06'
day = '02'

# Filename of the original results
filename_trade_ori = f'../data/itch_trade_classification_trade_{year}{month}{day}_AAPL_ori.txt'
filename_physical_ori = f'../data/itch_trade_classification_physical_{year}{month}{day}_AAPL_ori.txt'

# Filename of the corrected results
filename_trade_corr = f'../data/itch_trade_classification_trade_{year}{month}{day}_AAPL_corr.txt'
filename_physical_corr = f'../data/itch_trade_classification_physical_{year}{month}{day}_AAPL_corr.txt'

In [3]:
# Load and compute the data with my implementation
(time_signs, trade_signs,
 vol_signs, price_signs) = itch_sign_clas.itch_trade_classification_data(ticker, year, month, day)
id_trades = itch_sign_clas.itch_trade_classification_eq1_data(ticker, trade_signs, price_signs, year,
                                                              month, day)
emp_s, exp_eq2_s = itch_sign_clas.itch_trade_classification_eq2_data(ticker, time_signs,
                                                                              trade_signs, id_trades, year,
                                                                              month, day)
_, exp_eq3_s = itch_sign_clas.itch_trade_classification_eq3_data(ticker, time_signs, trade_signs,
                                                                    vol_signs, id_trades, year, month, day)

Processing data for the stock AAPL the 2008.06.02

Implementation of Eq. 1.
Processing data for the stock AAPL the 2008.06.02

Implementation of Eq. 2.
Processing data for the stock AAPL the 2008.06.02

Implementation of Eq. 3.
Processing data for the stock AAPL the 2008.06.02



In [4]:
# Create a data frame with my results
trades_no_0 = trade_signs != 0
d1 = {'Time': time_signs[trades_no_0], 'Emp': trade_signs[trades_no_0],
      'Exp': id_trades, 'Price': price_signs[trades_no_0]}
trades_juan_ori = pd.DataFrame(data=d1).astype(int)
trades_juan_ori['Price'] = trades_juan_ori['Price'] / 10000

In [5]:
d2 = {'Eq2': exp_eq2_s, 'Eq3': exp_eq3_s, 'Emp': emp_s}
physical_juan_ori = pd.DataFrame(data=d2).astype(int)

In [6]:
# Load results from paper
trades_paper_ori = pd.read_csv(filename_trade_ori, sep='   ',
                               usecols=(1,3,4,5), header=None, engine='python')
trades_paper_ori.columns = ['Time', 'Emp', 'Exp', 'Price']

In [7]:
physical_paper_ori = pd.read_csv(filename_physical_ori, sep='   ',
                               usecols=(2,3,4), header=None, engine='python')
physical_paper_ori.columns = ['Eq2', 'Eq3', 'Emp']

After loading the data, I compare every single result to find where are the differences

In [8]:
print('Similarities')
print('------------')
print()

# Trade time
trade_time_comp = np.sum(trades_paper_ori['Time'] == trades_juan_ori['Time']) \
                         / len(trades_paper_ori['Time'])
print('The similarity of the time in the trade time scale is {:.2f}%'.format(trade_time_comp * 100))
print()

# Empirical trade signs
trade_emp_comp = np.sum(trades_paper_ori['Emp'] == trades_juan_ori['Emp']) / len(trades_paper_ori['Emp'])
physical_emp_comp = np.sum(physical_paper_ori['Emp'] == physical_juan_ori['Emp']) \
                    / len(physical_paper_ori['Emp'])
print('The similarity of the empirical trade signs values for the trade time scale is  {:.2f}%'
      .format(trade_emp_comp * 100))
print('The similarity of the trade signs values for the physical time scale is {:.2f}%'
      .format(physical_emp_comp * 100))
print()

trade_exp_comp = np.sum(trades_paper_ori['Exp'] == trades_juan_ori['Exp']) / len(trades_paper_ori['Exp'])
physical_eq2_comp = np.sum(physical_paper_ori['Eq2'] == physical_juan_ori['Eq2']) \
                    / len(physical_paper_ori['Eq2'])
physical_eq3_comp = np.sum(physical_paper_ori['Eq3'] == physical_juan_ori['Eq3']) \
                    / len(physical_paper_ori['Eq3'])
print('The similarity of the experimental result for the trades is     {:.2f}%'
      .format(trade_emp_comp * 100))
print('The similarity of the Eq. 2 result for the physical time scale is {:.2f}%'
      .format(physical_eq2_comp * 100))
print('The similarity of the Eq. 3 result for the physical time scale is {:.2f}%'
      .format(physical_eq3_comp * 100)) 

Similarities
------------

The similarity of the time in the trade time scale is 100.00%

The similarity of the empirical trade signs values for the trade time scale is  100.00%
The similarity of the trade signs values for the physical time scale is 93.05%

The similarity of the experimental result for the trades is     100.00%
The similarity of the Eq. 2 result for the physical time scale is 93.04%
The similarity of the Eq. 3 result for the physical time scale is 91.13%


## Part 2. Find the error

In the trade time scale there is no difference between the paper results and my results.

In [9]:
trade_exp_diff = np.where(trades_paper_ori['Exp'] != trades_juan_ori['Exp'])[0]

print('The number of different values between the experimental results are ', len(trade_exp_diff))
print('The first ten different values are located in the positions ', trade_exp_diff[:10])

The number of different values between the experimental results are  1
The first ten different values are located in the positions  [0]


In [10]:
print('The first value of the paper is {} and the result of my implementation is {}'
      .format(trades_paper_ori['Exp'][0], trades_juan_ori['Exp'][0]))

The first value of the paper is -1 and the result of my implementation is 1


In [11]:
paper = 0
juan = 0

for val in trade_exp_diff:
    
    print('The value in pos. {} of the paper is {} and of my implementation is {}'
          .format(val, trades_paper_ori['Exp'][val], trades_juan_ori['Exp'][val]))
    print('To check which value is correct, we need to find the price in the position {} and {}'
          .format(val, val - 1))
    sign = np.sign(price_signs[val] - price_signs[val - 1])
    print('The price in pos. {} is {} and in position {} is {}. Then the trade sign must be {}'
              .format(val, price_signs[val], val - 1, price_signs[val - 1], sign))
    if (sign == 0):
        print(' '.join(('As the sign is zero, we use the trade sign value in pos. {}. In the paper the'
              + ' value is {} and for Juan that value is {}. The real value of that position is {}')
              .split()).format(val - 1, trades_paper_ori['Exp'][val - 1], trades_juan_ori['Exp'][val - 1],
              trade_signs[val - 1]))
        if (trades_paper_ori['Exp'][val - 1] == trade_signs[val - 1]
            and trades_juan_ori['Exp'][val - 1] == trade_signs[val - 1]):
            print('Both results were right')
            paper += 1
            juan += 1
        elif (trades_paper_ori['Exp'][val - 1] == trade_signs[val - 1]):
            print('The paper was right')
            paper += 1
        else:
            print('Juan was right')
            juan += 1
    else:
        print('The real value of that position is {}'.format(trade_signs[val]))
        if (trade_signs[val] == trades_paper_ori['Exp'][val]):
            print('The paper was right')
            paper += 1
        else:
            print('Juan was right')
            juan += 1
    print()

print('The paper was right {} times and Juan was right {} times'.format(paper, juan))    
print()

The value in pos. 0 of the paper is -1 and of my implementation is 1
To check which value is correct, we need to find the price in the position 0 and -1
The price in pos. 0 is 188.63 and in position -1 is 185.63. Then the trade sign must be 1.0
The real value of that position is -1.0
The paper was right

The paper was right 1 times and Juan was right 0 times



The reason of this error is due to the fact I just use the values from the open market time. Mi first value is calculated using the difference between the first and last value `diff = price_signs[0] - price_signs[-1]`, then my first value is error prone.

Now, I check the differences between the results obtained using eq. 2 in the paper and with my
implementation to find where is the error.

In [12]:
eq2_diff = np.where(physical_paper_ori['Eq2'] != physical_juan_ori['Eq2'])[0]

print('The total of different values between the experimental results are ', len(eq2_diff))
print('The first ten different values are located in the positions ', eq2_diff[:10])

The total of different values between the experimental results are  1545
The first ten different values are located in the positions  [ 0 38 46 53 63 72 74 77 86 91]


In [13]:
paper = 0
juan = 0
wrong = 0

for val in eq2_diff:
    
    print('The value in pos. {} of the paper is {} and of my implemantation is {}'
          .format(val, physical_paper_ori['Eq2'][val], physical_juan_ori['Eq2'][val]))
    print('To check which value is correct, we need to find the value in the second {}'
          .format(34800 + val))
    
    condition = (trades_paper_ori['Time'] / 1000 >= 34800 + val) \
                * (trades_paper_ori['Time'] / 1000 < 34800 + val + 1)
    print('Paper: ', list(trades_paper_ori['Exp'][condition]))
    print('Juan: ', list(trades_juan_ori['Exp'][condition]))
        
    trades_sign_eq2_juan = np.sum(trades_juan_ori['Exp'][condition])
    trades_sign_eq2_paper = np.sum(trades_paper_ori['Exp'][condition])
    trades_sign_eq2_emp = np.sum(trades_paper_ori['Emp'][condition])
    
    print('The sum of Juan is {}, the sum of the paper is {}, the empirical value is {}'
          .format(trades_sign_eq2_juan, trades_sign_eq2_paper, np.sign(trades_sign_eq2_emp)))                   

    if (np.sign(trades_sign_eq2_juan) == np.sign(trades_sign_eq2_emp)):
        print('Juan was right')
        juan += 1
    elif (np.sign(trades_sign_eq2_paper) == np.sign(trades_sign_eq2_emp)):
        print('The paper was right')
        paper += 1
    else:
        print('Both were wrong')
        wrong +=1

    print()

print('The paper was right {} times, Juan was right {} times and both were wrong {} times'
      .format(paper, juan, wrong))    
print()

The value in pos. 0 of the paper is -1 and of my implemantation is 0
To check which value is correct, we need to find the value in the second 34800
Paper:  [-1, 1, -1, -1]
Juan:  [1, 1, -1, -1]
The sum of Juan is 0, the sum of the paper is -2, the empirical value is -1
The paper was right

The value in pos. 38 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 34838
Paper:  [-1, -1, -1, -1]
Juan:  [-1, -1, -1, -1]
The sum of Juan is -4, the sum of the paper is -4, the empirical value is 1
Both were wrong

The value in pos. 46 of the paper is 1 and of my implemantation is 0
To check which value is correct, we need to find the value in the second 34846
Paper:  [-1, 1]
Juan:  [-1, 1]
The sum of Juan is 0, the sum of the paper is 0, the empirical value is 1
Both were wrong

The value in pos. 53 of the paper is 0 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 34853


/home/tp/jchenaol/.pyenv/versions/3.6.5/lib/python3.6/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Paper:  [1, 1]
Juan:  [1, 1]
The sum of Juan is 2, the sum of the paper is 2, the empirical value is 0
Both were wrong

The value in pos. 368 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 35168
Paper:  [-1]
Juan:  [-1]
The sum of Juan is -1, the sum of the paper is -1, the empirical value is 1
Both were wrong

The value in pos. 446 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 35246
Paper:  [-1, -1, -1, -1, 1, 1, 1]
Juan:  [-1, -1, -1, -1, 1, 1, 1]
The sum of Juan is -1, the sum of the paper is -1, the empirical value is 1
Both were wrong

The value in pos. 468 of the paper is -1 and of my implemantation is 0
To check which value is correct, we need to find the value in the second 35268
Paper:  [1, -1]
Juan:  [1, -1]
The sum of Juan is 0, the sum of the paper is 0, the empirical value is -1
Both were wrong

The value in pos. 474 of the 

The sum of Juan is 1, the sum of the paper is 1, the empirical value is -1
Both were wrong

The value in pos. 738 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 35538
Paper:  [1, 1, -1, -1, -1, 1, -1]
Juan:  [1, 1, -1, -1, -1, 1, -1]
The sum of Juan is -1, the sum of the paper is -1, the empirical value is 1
Both were wrong

The value in pos. 763 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 35563
Paper:  [-1]
Juan:  [-1]
The sum of Juan is -1, the sum of the paper is -1, the empirical value is 1
Both were wrong

The value in pos. 776 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 35576
Paper:  [1]
Juan:  [1]
The sum of Juan is 1, the sum of the paper is 1, the empirical value is -1
Both were wrong

The value in pos. 782 of the paper is 1 and of my implemantation 

The sum of Juan is -2, the sum of the paper is -2, the empirical value is 0
Both were wrong

The value in pos. 1198 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 35998
Paper:  [-1, -1, -1]
Juan:  [-1, -1, -1]
The sum of Juan is -3, the sum of the paper is -3, the empirical value is 1
Both were wrong

The value in pos. 1211 of the paper is 1 and of my implemantation is 0
To check which value is correct, we need to find the value in the second 36011
Paper:  [-1, 1]
Juan:  [-1, 1]
The sum of Juan is 0, the sum of the paper is 0, the empirical value is 1
Both were wrong

The value in pos. 1212 of the paper is 1 and of my implemantation is 0
To check which value is correct, we need to find the value in the second 36012
Paper:  [-1, -1, 1, 1]
Juan:  [-1, -1, 1, 1]
The sum of Juan is 0, the sum of the paper is 0, the empirical value is 1
Both were wrong

The value in pos. 1213 of the paper is 0 and of my implemantatio

...

Paper:  [-1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1]
Juan:  [-1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1]
The sum of Juan is -6, the sum of the paper is -6, the empirical value is 1
Both were wrong

The value in pos. 20711 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 55511
Paper:  [1, 1]
Juan:  [1, 1]
The sum of Juan is 2, the sum of the paper is 2, the empirical value is -1
Both were wrong

The value in pos. 20712 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 55512
Paper:  [1]
Juan:  [1]
The sum of Juan is 1, the sum of the paper is 1, the empirical value is -1
Both were wrong

The value in pos. 20716 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 55516
Paper:  [1, 1, 1]
Juan:  [1, 1, 1]
The sum of Juan is 3, the sum of the paper is 3, the empiri

Juan:  [1, 1, 1]
The sum of Juan is 3, the sum of the paper is 3, the empirical value is -1
Both were wrong

The value in pos. 20968 of the paper is 0 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 55768
Paper:  [1, 1, 1, 1]
Juan:  [1, 1, 1, 1]
The sum of Juan is 4, the sum of the paper is 4, the empirical value is 0
Both were wrong

The value in pos. 20971 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 55771
Paper:  [-1, -1, 1]
Juan:  [-1, -1, 1]
The sum of Juan is -1, the sum of the paper is -1, the empirical value is 1
Both were wrong

The value in pos. 20973 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 55773
Paper:  [1, 1]
Juan:  [1, 1]
The sum of Juan is 2, the sum of the paper is 2, the empirical value is -1
Both were wrong

The value in pos. 20978 of the paper is -1 and of m

Paper:  [1, 1, -1]
Juan:  [1, 1, -1]
The sum of Juan is 1, the sum of the paper is 1, the empirical value is -1
Both were wrong

The value in pos. 21337 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 56137
Paper:  [1, 1]
Juan:  [1, 1]
The sum of Juan is 2, the sum of the paper is 2, the empirical value is -1
Both were wrong

The value in pos. 21341 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 56141
Paper:  [-1, -1]
Juan:  [-1, -1]
The sum of Juan is -2, the sum of the paper is -2, the empirical value is 1
Both were wrong

The value in pos. 21342 of the paper is 0 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 56142
Paper:  [1, 1]
Juan:  [1, 1]
The sum of Juan is 2, the sum of the paper is 2, the empirical value is 0
Both were wrong

The value in pos. 21352 of the paper is 0 and of 


The value in pos. 21721 of the paper is 0 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 56521
Paper:  [-1, -1]
Juan:  [-1, -1]
The sum of Juan is -2, the sum of the paper is -2, the empirical value is 0
Both were wrong

The value in pos. 21734 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 56534
Paper:  [1, 1, 1]
Juan:  [1, 1, 1]
The sum of Juan is 3, the sum of the paper is 3, the empirical value is -1
Both were wrong

The value in pos. 21741 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 56541
Paper:  [-1]
Juan:  [-1]
The sum of Juan is -1, the sum of the paper is -1, the empirical value is 1
Both were wrong

The value in pos. 21746 of the paper is -1 and of my implemantation is 1
To check which value is correct, we need to find the value in the second 56546
Paper:  [1, 1, 1, 1,

The sum of Juan is 2, the sum of the paper is 2, the empirical value is 0
Both were wrong

The value in pos. 21990 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 56790
Paper:  [1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1]
Juan:  [1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1]
The sum of Juan is -4, the sum of the paper is -4, the empirical value is 1
Both were wrong

The value in pos. 22000 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 56800
Paper:  [-1, -1, -1, -1, 1, 1]
Juan:  [-1, -1, -1, -1, 1, 1]
The sum of Juan is -2, the sum of the paper is -2, the empirical value is 1
Both were wrong

The value in pos. 22007 of the paper is 1 and of my implemantation is -1
To check which value is correct, we need to find the value in the second 56807
Paper:  [-1, -1]
Juan:  [-1, -1]
The sum of Juan is -2, the sum of the paper is -2, the empirical val

According to the comparison, the error between the two implementations are due to the classification.

## Interpretation problem in the paper

As the trade signs from the trade time scale are the same for the paper and for my implementation, and as
the physical time scale results are based on the trade time scale classification, the mistake must be in the
classification of the physical time scale results.

Reviewing thoroughly the implementation of the paper, I found a detail that in my opinion is a mistake.
The paper uses the empirical results as the experimental results, and the classification results as the
theoretical results.

Since the empirical values are the values that we want to replicate, they must be the "ground truth" and
the classification results have to be compared with the empirical results.

Even, in Eq. 3 implementation were the empirical values the ones used to be multiplied by the volume, which make no sense.

To correct this, I changed the paper code and obtained the "correct" data. With that this are the new results of the comparison

In [14]:
# Load new data
trades_paper_corr = pd.read_csv(filename_trade_corr, sep='   ',
                                usecols=(1,3,4,5), header=None, engine='python')
trades_paper_corr.columns = ['Time', 'Emp', 'Exp', 'Price']

physical_paper_corr = pd.read_csv(filename_physical_corr, sep='   ',
                                usecols=(2,3,4), header=None, engine='python')
physical_paper_corr.columns = ['Eq2', 'Eq3', 'Emp']

In [15]:
# Comparison

trade_time_comp = np.sum(trades_paper_corr['Time'] == trades_juan_ori['Time']) \
                  / len(trades_paper_corr['Time'])
print('The similarity of the time in the trade time scale is {:.2f}%'.format(trade_time_comp * 100))
print()

trades_emp_comp = np.sum(trades_paper_corr['Emp'] == trades_juan_ori['Emp']) \
                  / len(trades_paper_corr['Emp'])
physical_emp_comp = np.sum(physical_paper_corr['Emp'] == physical_juan_ori['Emp']) \
                  / len(physical_paper_corr['Emp'])
print('The similarity of the trade signs values for the trade time scale is  {:.2f}%'
      .format(trades_emp_comp * 100))
print('The similarity of the trade signs values for the physical time scale is {:.2f}%'
      .format(physical_emp_comp * 100))
print()

trades_exp_comp = np.sum(trades_paper_corr['Exp'] == trades_juan_ori['Exp']) \
                  / len(trades_paper_corr['Exp'])
physical_eq2_comp = np.sum(physical_paper_corr['Eq2'] == physical_juan_ori['Eq2']) \
                  / len(physical_paper_corr['Eq2'])
physical_eq3_comp = np.sum(physical_paper_corr['Eq3'] == physical_juan_ori['Eq3']) \
                  / len(physical_paper_corr['Eq3'])
print('The similarity of the experimental result for the trade time scale is {:.2f}%'
      .format(trade_exp_comp * 100))
print('The similarity of the Eq. 2 result for the physical time scale is       {:.2f}%'
      .format(physical_eq2_comp * 100))
print('The similarity of the Eq. 3 result for the physical time scale is       {:.2f}%'
      .format(physical_eq3_comp * 100))

The similarity of the time in the trade time scale is 100.00%

The similarity of the trade signs values for the trade time scale is  100.00%
The similarity of the trade signs values for the physical time scale is 100.00%

The similarity of the experimental result for the trade time scale is 100.00%
The similarity of the Eq. 2 result for the physical time scale is       100.00%
The similarity of the Eq. 3 result for the physical time scale is       96.47%
